Binary classification techniques work well when the data observations belong to one of two classes or categories, such as *true* or *false*. When the data can be categorized into more than two classes, you must use a *multiclass classification* algorithm.

### Meet the data

In this exercise, you'll build a multiclass classifier to separate penguins into categories of species.

You'll be using *palmerpenguins* data, which contains size measurements for three penguin species that were observed on three islands in the Palmer Archipelago, Antarctica.

> **Citation**: The *palmerpenguins* data was collected from 2007 to 2009 by Dr. Kristen Gorman with the [Palmer Station Long Term Ecological Research Program](https://pal.lternet.edu/), part of the [US Long Term Ecological Research Network](https://lternet.edu/). The data was imported directly from the [Environmental Data Initiative](https://environmentaldatainitiative.org/) (EDI) data portal, and it's available for use by CC0 license ("No Rights Reserved") in accordance with the [Palmer Station Data Policy](https://pal.lternet.edu/data).

In R, the *palmerpenguins* package, by [Allison Marie Horst and Alison Presmanes Hill and Kristen B Gorman](https://allisonhorst.github.io/palmerpenguins/articles/intro.html), provides the data that's related to these adorable creatures.

The CSV dataset that's used in the corresponding [Python learning module](https://learn.microsoft.com/en-us/learn/modules/train-evaluate-classification-models/) can be found in the [penguins.csv file](https://github.com/MicrosoftDocs/ml-basics/tree/master/data) on GitHub.

Care for a pun? 

**Q**: What is a penguin's favorite movie?

**A**: Frozen.

With that, let's get started exploring some penguin data.

### Explore the data


In [ ]:
# Load the required packages and make them available in your current R session
suppressPackageStartupMessages({
  library(tidyverse)
  library(palmerpenguins)
  library(tidymodels)
})

# Take a peek into the data
glimpse(penguins)


The data contains the following columns:

-   **species**: The name of the penguin species (Adelie, Chinstrap, Gentoo)

-   **island**: The name of the island where the penguins were observed (in Palmer Archipelago, Antarctica)

-   **bill_length_mm (also called culmen_length)**: The length of the dorsal ridge of the penguin's bill (millimeters)

-   **bill_depth_mm (also called culmen_depth)**: The depth of the penguin's bill (millimeters)

-   **flipper_length_mm**: The length of the penguin's flippers (millimeters)

-   **body_mass_g**: The penguin's body mass (grams)

-   **sex**: The penguin's sex (male, female)

-   **year**: The study year (2007, 2008, or 2009)

The **species** column, which contains penguin species Adelie, Chinstrap, or Gentoo, is the label that you want to train a model to predict.

The corresponding [Python tutorial](https://learn.microsoft.com/en-us/learn/modules/train-evaluate-classification-models/7-exercise-multiclass-classification) uses a dataset with the following variables: *bill_length_mm*, *bill_depth_mm*, *flipper_length_mm*, *body_mass_g*, *species*.

In this exercise, let's narrow the scope and use only those labels, and then make some summary statistics while we're at it. The [skimr package](https://cran.r-project.org/web/packages/skimr/vignettes/skimr.html) provides a strong set of summary statistics that are generated for a variety of data types.


In [ ]:
# Load the skimr column
library(skimr)

# Select desired columns
penguins_select <- penguins %>% 
  select(c(bill_length_mm, bill_depth_mm, flipper_length_mm,
           body_mass_g, species))

# Do some summary statistics
penguins_select %>% 
  skim()


From the neat summary provided by skimr, you can see that each of the predictor columns has two missing values, and the label/outcome column has none.

Now, dig a little deeper and filter the rows that contain missing values.


In [ ]:
penguins_select %>% 
  filter(if_any(everything(), is.na))


Two of the rows contain no feature values at all (*NA* stands for *not available*). These rows won't be useful in training a model, so you can discard them from the dataset.



In [ ]:
# Drop rows containing missing values
penguins_select <- penguins_select %>% 
  drop_na()

# Confirm there are no missing values
penguins_select %>% 
  anyNA()

# Proportion of each species in the data
penguins_select %>% 
  count(species)


Now that you've dealt with the missing values, let's explore how the features relate to the label. 

Start by creating some box charts, as shown here:



In [ ]:
# Load the paletteer package
library(paletteer)

# Pivot data to a long format
penguins_select_long <- penguins_select %>% 
  pivot_longer(!species, names_to = "predictors", values_to = "values")

# Make box plots
theme_set(theme_light())
penguins_select_long %>%
  ggplot(mapping = aes(x = species, y = values, fill = predictors)) +
  geom_boxplot() +
  facet_wrap(~predictors, scales = "free") +
  scale_fill_paletteer_d("nbapalettes::supersonics_holiday") +
  theme(legend.position = "none")
  


In the box plots, it appears that the Adelie and Chinstrap species have similar data profiles for bill_depth, flipper_length, and body_mass, but Chinstrap tends to have longer bill_length. Gentoo features tend to be fairly clearly differentiated from the others, which should help you train a good classification model.

### Prepare the data

Just as for binary classification, before you train the model, you need to split the data into subsets for training and validation. To maintain the proportion of each label value in the training and validation datasets, you'll also apply a *stratification* technique when you split the data.


In [ ]:
# For reproducibility
set.seed(2056)

# Split data 70%-30% into training set and test set
penguins_split <- penguins_select %>% 
  initial_split(prop = 0.70, strata = species)

# Extract data in each split
penguins_train <- training(penguins_split)
penguins_test <- testing(penguins_split)

# Print the number of observations in each split
cat("Training cases: ", nrow(penguins_train), "\n",
    "Test cases: ", nrow(penguins_test), sep = "")


### Train and evaluate a multiclass classifier

Now that you have a set of training features and corresponding training labels, you can fit a multiclass classification algorithm to the data to create a model.

The function `parsnip::multinom_reg()` defines a model that uses linear predictors to predict multiclass data by using the multinomial distribution.

Let's fit multinomial regression via the [nnet](https://cran.r-project.org/web/packages/nnet/nnet.pdf) package. This model usually has one tuning hyperparameter, `penalty`, which describes the amount of regularization. You use it to counteract any bias in the sample and help the model generalize well by avoiding *overfitting* the model to the training data. You can of course tune this parameter, as you'll do later in this exercise but, for now, let's choose an arbitrary value of *1*.


In [ ]:
# Specify a multinomial regression via nnet
multireg_spec <- multinom_reg(penalty = 1) %>% 
  set_engine("nnet") %>% 
  set_mode("classification")

# Train a multinomial regression model without any preprocessing
set.seed(2056)
multireg_fit <- multireg_spec %>% 
  fit(species ~ ., data = penguins_train)

# Print the model
multireg_fit


Now you can use the trained model to predict the labels for the test features and evaluate performance:



In [ ]:
# Make predictions for the test set
penguins_results <- penguins_test %>% select(species) %>% 
  bind_cols(multireg_fit %>% 
              predict(new_data = penguins_test)) %>% 
  bind_cols(multireg_fit %>% 
              predict(new_data = penguins_test, type = "prob"))

# Print predictions
penguins_results %>% 
  slice_head(n = 5)


Let's look at the confusion matrix for our model.



In [ ]:
# Confusion matrix
penguins_results %>% 
  conf_mat(species, .pred_class)


The confusion matrix shows the intersection of predicted and actual label values for each class. In simpler terms, the diagonal intersections from top-left to bottom-right indicate the number of correct predictions.

When you're dealing with multiple classes, it's ordinarily more intuitive to visualize this output as a heat map, like this:


In [ ]:
update_geom_defaults(geom = "tile", new = list(color = "black", alpha = 0.7))
# Visualize confusion matrix
penguins_results %>% 
  conf_mat(species, .pred_class) %>% 
  autoplot(type = "heatmap")


The darker squares in the confusion matrix indicate high numbers of cases and, hopefully, you can see that they form a diagonal line that shows where the predicted and actual labels are the same.

Calculate summary statistics for the confusion matrix, as shown here:


In [ ]:
# Statistical summaries for the confusion matrix
conf_mat(data = penguins_results, truth = species, estimate = .pred_class) %>% 
  summary()


The tibble shows the overall metrics of how well the model performs across all three classes.

Now let's evaluate the receiver operating characteristic (ROC) metrics. In the case of a multiclass classification model, a single ROC curve shows that a true positive rate versus false positive rate is not possible. However, you can use the rates for each class in a One-vs-Rest (OVR) comparison to create an ROC chart for each class.


In [ ]:
# Make an ROC_CURVE
penguins_results %>% 
  roc_curve(species, c(.pred_Adelie, .pred_Chinstrap, .pred_Gentoo)) %>% 
  ggplot(aes(x = 1 - specificity, y = sensitivity, color = .level)) +
  geom_abline(lty = 2, color = "gray80", size = 0.9) +
  geom_path(show.legend = T, alpha = 0.6, size = 1.2) +
  coord_equal()


That went down well! The model did a great job in classifying the penguins.

What kind of adventure would it be if you didn't preprocess the data for modeling?

### Workflows with a support vector machine algorithm

Again, just as with binary classification, you can use a workflow to apply preprocessing steps to the data before you fit it to an algorithm to train a model. Let's scale the numeric features in a transformation step before training. Just to show that you can, try a different algorithm (a *support vector machine* [SVM]) and tune some model hyperparameters.

Support vector machine algorithms try to find a *hyperplane* in some feature space that best separates the classes. To learn more, see:

- [Hands-on machine learning with R](https://bradleyboehmke.github.io/HOML/svm.html)
- [An introduction to statistical learning with applications in R](https://www.statlearning.com/)
- [Support vector machines under the hood](https://towardsdatascience.com/support-vector-machines-under-the-hood-c609e57a4b09)
- [SVM kernels](https://scikit-learn.org/stable/auto_examples/svm/plot_svm_kernels.html)

You'll fit a *radial basis function* support vector machine to this data and tune the *SVM cost parameter* and the *σ parameter* in the kernel function (the margin parameter doesn't apply to classification models).

-   When you use a cost argument, you can specify the cost of a violation to the margin. When the cost argument is small, the margins are wide, and many support vectors will be on the margin or will violate the margin. This *could* make the model more robust and lead to better classification. When the cost argument is large, the margins will be narrow, and there will be few support vectors on the margin or that violate the margin.

- As *σ* decreases, the fit becomes more nonlinear, and the model becomes more flexible.

Both parameters can have a profound effect on the model complexity and performance.

The radial basis kernel is extremely flexible. For this reason, in practice, you would ordinarily start with this kernel when you're fitting SVMs.

You mark parameters for tuning by assigning them a value of `tune()`.

Also, how about trying out a new succinct way to create workflows that minimizes a lot of piping steps, as shown here:


In [ ]:
# Create a model specification
svm_spec <- svm_rbf(mode = "classification", engine = "kernlab",
            cost = tune(), rbf_sigma = tune())


# Create a workflow that encapsulates a recipe and a model
svm_wflow <- recipe(species ~ ., data = penguins_train) %>% 
  step_normalize(all_numeric_predictors()) %>% 
  workflow(svm_spec)

# Print out workflow
svm_wflow


Pretty neat, right?

Now that you've specified what parameter to tune, you need to figure out a set of possible values to try out and then choose the best.

To do this, you create a grid. In this example, you work through a regular grid of hyperparameter values, try them out, and see what pair results in the best model performance.


In [ ]:
set.seed(2056)
# Create a regular grid of six values for each tuning parameter
svm_grid <- grid_regular(parameters(svm_spec), levels = 6)

# Print out some parameters in your grid
svm_grid %>% 
  slice_head(n = 10)


Awesome! One thing about hyperparameters is that they're not learned directly from the training set. Instead, you estimate them by using *simulated datasets* that are created from a process called *resampling*.

Let's try out a *bootstrap resampling* technique. Bootstrap resampling means that you draw with a *replacement* from your original dataset, fit a model on that new set, which contains some duplicates, and then evaluate the model on the data points that weren't included.

Then you repeat the process. The default behavior is 25 bootstraps, but this can be changed. 

Okay, let's create some simulated datasets.


In [ ]:
set.seed(2056)
# Bootstrap resampling
penguins_bs <- bootstraps(penguins_train, times = 10)

penguins_bs


### Tune your model via grid search

You're ready to tune. Use [`tune_grid()`](https://tune.tidymodels.org/reference/tune_grid.html) to fit models at all the values you chose for each hyperparameter.


In [ ]:
doParallel::registerDoParallel()

# Model tuning via a grid search
set.seed(2056)
svm_res <- tune_grid(
  object = svm_wflow,
  resamples = penguins_bs,
  grid = svm_grid
)


Now that you have your tuning results, you can extract the performance metrics by using `collect_metrics()`:



In [ ]:
# Obtain performance metrics
svm_res %>% 
  collect_metrics() %>% 
  slice_head(n = 7)


Let's see if you can get more by visualizing the results:



In [ ]:
# Visualize the tuning metrics
svm_res %>% 
  collect_metrics() %>% 
  mutate(rbf_sigma = factor(rbf_sigma)) %>% 
  ggplot(mapping = aes(x = cost, y = mean, color = rbf_sigma)) +
  geom_line(size = 1.5, alpha = 0.7) +
  geom_point(size = 2) +
  facet_wrap(~.metric, scales = "free", nrow = 2) +
  scale_x_log10(labels = scales::label_number()) +
  scale_color_viridis_d(option = "viridis", begin = .1)


It seems that an SVM with an *rbf_sigma* of 1 and 0.01 performed really well across all candidate values of cost. The [`show_best()`](https://tune.tidymodels.org/reference/show_best.html) function can help you make a clearer distinction:



In [ ]:
# Show best submodel 
svm_res %>% 
  show_best("accuracy")


Much better! Now, use the [`select_best()`](https://tune.tidymodels.org/reference/show_best.html) function to pull out the single set of hyperparameter values in the best submodel:



In [ ]:
# Select best model hyperparameters
best_svm <- svm_res %>% 
  select_best("accuracy")

best_svm


Perfect! These are the values for *rbf_sigma* and *cost* that maximize the accuracy of your penguin classification exercise.

You can now finalize your workflow, so that the parameters you had marked for tuning by assigning them a value of `tune()` can get updated with the values from `best_svm`.


In [ ]:
# Finalize the workflow
final_wflow <- svm_wflow %>% 
  finalize_workflow(best_svm)

final_wflow
  


That marks the end of tuning.

### The last fit

Finally, by using your test data, fit this final model to the training data and evaluate how it would perform on new data. You can use the function [`last_fit()`](https://tune.tidymodels.org/reference/last_fit.html) with your finalized model. The function *fits* the finalized model on the full training dataset and *evaluates* it on the testing data.


In [ ]:
# The last fit
final_fit <- last_fit(object = final_wflow, split = penguins_split)

# Collect metrics
final_fit %>% 
  collect_metrics()


Much better! You can go ahead and obtain the hard class and probability predictions by using `collect_predictions()`, and you'll be well on your way to computing the confusion matrix and other summaries that come with it.



In [ ]:
# Collect predictions and make a confusion matrix
final_fit %>% 
  collect_predictions() %>% 
  conf_mat(truth = species, estimate = .pred_class)


### Use the model with new data observations

Now, save your trained model so that you can use it again later. Begin by extracting the *trained workflow* object from the `final_fit` object.


In [ ]:
# Extract the trained workflow
penguins_svm_model <- final_fit %>% 
  extract_workflow()

# Save the workflow
library(here)
saveRDS(penguins_svm_model, "penguins_svm_model.rds")


Okay, so now you have a trained model. Use it to predict the class of a new penguin observation:



In [ ]:
# Load the model
loaded_psvm_model <- readRDS("penguins_svm_model.rds") 

# Create a new tibble of observations
new_obs <- tibble(
  bill_length_mm = c(49.5, 38.2),
  bill_depth_mm = c(18.4, 20.1),
  flipper_length_mm = c(195, 190),
  body_mass_g = c(3600, 3900))

# Make predictions
new_results <- new_obs %>% 
  bind_cols(loaded_psvm_model %>% 
              predict(new_data = new_obs))

# Show predictions
new_results


Good job! You now have a working model.

### Summary

Classification is one of the most common forms of machine learning. By following the basic principles we've discussed in this exercise, you should be able to train, evaluate, and tune classification models with tidymodels. It's worth spending some time investigating classification algorithms in more depth, and a good starting point is the [tidymodels documentation](https://www.tidymodels.org/).

